The decision tree is a very intuitive way of representing hierarchical (i.e. nonlinear) relationships. The general idea of the of the algorithm is: split the observations along the predictors in such a way that that the splitted observations as dissimilar as possible along the targets. In other words we have to find a way to discriminate by X in order to separate Y as much as possible.

Short glossary:
Root node — node at the top of the tree. 
Decision nodes — nodes where the variables are evaluated. These nodes have arrows pointing to them and away from them
Leaf nodes — final node where the prediction is made

What we will need to implement the algorithm:
1. Choose an improvement metric (it is usually called node purness) for the alogrithm, the common are: information gain, gain ratio, or Gini index.
2. Split the data to maximize the improvement metric. 
3. Stop when the end node is "pure" or a stopping condition is reach. Common stopping conditions are: tree depth,  minmum sample size in a given leaf.

Step 2 is reiterated until step 3 is reached

Lets's use entropy and information gain as our metric. Entropy is defined as:  
$H(X) = \mathop{\mathbb{E}}[-log_2(p(x))] = -\Sigma_nlog_2(p_i(x))$

In [1]:
import numpy as np
from collections import Counter

def calc_entropy(arr):
    freq = np.bincount(arr)
    prob = freq / len(arr)
    
    entropy = 0
    for p in prob:
        if p > 0:
            entropy += p * np.log2(p)
    return -entropy


In [2]:
(1+1.8/100) * (1+6.20/100) * (1+3.00/100)

1.11354948

In [3]:
#lets do some caluclations with this metric
#pure array = no entropy
pure_array = [0, 0, 0, 0]
print(f'Entropy: {np.round(calc_entropy(pure_array), 3)}')

Entropy: -0.0


In [4]:
#impure array = max entropy
impure_array = [1, 2, 3, 0]
print(f'Entropy: {np.round(calc_entropy(impure_array), 3)}')

Entropy: 2.0


In [5]:
#mixed array = some entropy
mixed_array1 = [1, 1, 0, 0]
print(f'Entropy: {np.round(calc_entropy(mixed_array1), 3)}')

mixed_array2 = [1, 0, 0, 0]
print(f'Entropy: {np.round(calc_entropy(mixed_array2), 3)}')

Entropy: 1.0
Entropy: 0.811


The algorithm should strive to decrease entropy as much as possible and we can measure this by calculating information gain $IG(T, a) = H(T) - H(T|a)$  
Let's just call $H(T)$ the entropy of parent node and  H(T|a) the average of child nodes after the split. And thus without delving a lot into the intricacies of information gain we have a measure of improvement - the more the entropy falls after each split the better the split is.

In [6]:
def calc_information_gain(parent, child1, child2):
    weight1 = len(child1) / len(parent)
    weight2 = len(child2) / len(parent)
    
    ig = calc_entropy(parent) - (weight1 * calc_entropy(child1) + weight2 * calc_entropy(child2))
    return ig

In [7]:
#let's calculate a very simple case
#split this parent into these children
parent1 = [1, 1, 1, 1, 1, 0, 0, 0, 0, 0]
child1 =  [0, 0, 0, 1, 0]
child2 =  [1, 1, 1, 0, 1]

In [8]:
calc_information_gain(parent1, child1, child2)

0.2780719051126377

In [9]:
#let's check our work
print(calc_entropy(parent1))
print(calc_entropy(child1))
print(calc_entropy(child2))

1.0
0.7219280948873623
0.7219280948873623


In [10]:
calc_entropy(parent1)  - (calc_entropy(child1) + calc_entropy(child2))/2

0.2780719051126377

Ok now we need to get use the information gain to find the best split for the data. Let's start with a simple case and load the iris dataset.

In [11]:
import pandas as pd
from sklearn import datasets
from scipy.stats import mode

iris = datasets.load_iris()

#prepping the input data, let's leave the
df = pd.DataFrame(data=iris.data, columns=iris.feature_names)
df['target'] = iris.target
X = df.drop('target', axis=1)
y = df[['target']]

Let's inspct the data

In [12]:
X.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


In [13]:
y.head()

,target
0,0
1,0
2,0
3,0
4,0


Now what we need to do is to split the $y$ by each predictor ($n_pred$) and by each unique value of the predictor (we will call this $threshold$)

In [14]:
X_np=X.to_numpy()
y_np=y.to_numpy()

In [15]:
class Node:
    '''
    Helper class which implements a single tree node.
    '''
    def __init__(self, feature=None, threshold=None, data_left=None, data_right=None, gain=None, value=None):
        self.feature = feature
        self.threshold = threshold
        self.data_left = data_left
        self.data_right = data_right
        self.gain = gain
        self.value = value

In [17]:
def get_best_split(X, y):
    best_split = {}
    best_info_gain = -1
    n_rows, n_cols = X.shape

    df = np.concatenate((X, y.reshape(1, -1).T), axis=1)
    # For every dataset feature
    for f_idx in range(n_cols):
        X_curr = X[:, f_idx]
        # For every unique value of that feature
        for threshold in np.unique(X_curr):
            # Construct a dataset and split it to the left and right parts
            # Left part includes records lower or equal to the threshold
            # Right part includes records higher than the threshold
            
            df_left = np.array([row for row in df if row[f_idx] <= threshold])
            df_right = np.array([row for row in df if row[f_idx] > threshold])

            # Do the calculation only if there's data in both subsets
            if len(df_left) > 0 and len(df_right) > 0:
                # Obtain the value of the target variable for subsets
                y = df[:, -1].astype('int64')
                y_left = df_left[:, -1].astype('int64')
                y_right = df_right[:, -1].astype('int64')

                # Caclulate the information gain and save the split parameters
                # if the current split if better then the previous best
                #import ipdb; ipdb.set_trace()
                gain = calc_information_gain(y, y_left, y_right)
                if gain > best_info_gain:
                    best_split = {
                        'predictor_i': f_idx,
                        'threshold': threshold,
                        'data_left': df_left,
                        'data_right': df_right,
                        'gain': gain
                    }
                    best_info_gain = gain
    return(best_split)

In [19]:
b=get_best_split(X_np, y_np)

In [21]:
#Let's get the first split
split_1 = get_best_split(X_np, y_np)
split_1['gain']

0.9182958340544894

In [22]:
#split_1

In [23]:
#now lets split it again
#splitting child1 is pointless because we alrady have min entropy
split_1_1 = get_best_split(split_1['child1'][:,:-1], split_1['child1'][:,[-1]])
print(split_1_1['gain'])

0.0


In [24]:
split_1_2 = get_best_split(split_1['child2'][:,:-1], split_1['child2'][:,[-1]])
print(split_1_2['gain'])

0.6901603707546748


In [25]:
split_1_2_1 = get_best_split(split_1_2['child2'][:,:-1], split_1_2['child2'][:,[-1]])

In [26]:
#Negative gain was achieved, splitting here is pointless
split_1_2_1['gain']

0.09120811177442958

We could on and on but we already see that no gain was achieved in the last split, so we need some stopping condition. Usually we use these
1. Positive information gain
2. Max depth of the tree
3. Minimum number of observation in a split

Ok we have the core of the algorithm now we need it to bring it together. We need to apply `get_best_split` until any of the above conditoins are met for that we will use recursion. 

In [21]:
from scipy import stats as st
def grow_tree(X, y, depth=0, min_samples_split=2, max_depth=3):
    n_rows, n_cols = X.shape

    # Check to see if a node should be leaf node
    if n_rows >= min_samples_split and depth <= max_depth:
        # Get the best split
        best = get_best_split(X, y)
        #import ipdb;ipdb.set_trace()
        # If the split isn't pure
        if best['gain'] > 0:
            child1 = grow_tree(
                X=best['data_left'][:, :-1], 
                y=best['data_left'][:, -1], 
                depth=depth + 1
            )
            child2 = grow_tree(
                X=best['data_right'][:, :-1], 
                y=best['data_right'][:, -1], 
                depth=depth + 1
            )

            node = Node(
                feature=best['predictor_i'], 
                threshold=best['threshold'], 
                data_left=child1, 
                data_right=child2, 
                gain=best['gain']
            )

            return node
    #Leaf node - predict mode
    return Node(value=st.mode(y)[0][0])

In [22]:
def traverse_tree(tree):
    if tree:
        print(vars(tree))
        traverse_tree(tree.data_left)
        traverse_tree(tree.data_right)

In [23]:
my_tree=grow_tree(X_np, y_np, depth = 0, min_samples_split=5, max_depth=4)

In [24]:
traverse_tree(my_tree)

{'feature': 2, 'threshold': 1.9, 'data_left': <__main__.Node object at 0x7f98e028ad70>, 'data_right': <__main__.Node object at 0x7f98e03e81c0>, 'gain': 0.9182958340544894, 'value': None}
{'feature': None, 'threshold': None, 'data_left': None, 'data_right': None, 'gain': None, 'value': 0.0}
{'feature': 3, 'threshold': 1.7, 'data_left': <__main__.Node object at 0x7f98e0454580>, 'data_right': <__main__.Node object at 0x7f98e03ea560>, 'gain': 0.6901603707546748, 'value': None}
{'feature': 2, 'threshold': 4.9, 'data_left': <__main__.Node object at 0x7f98e025a0e0>, 'data_right': <__main__.Node object at 0x7f98e0455240>, 'gain': 0.21317043093799645, 'value': None}
{'feature': 3, 'threshold': 1.6, 'data_left': <__main__.Node object at 0x7f99b057afb0>, 'data_right': <__main__.Node object at 0x7f99b0578d30>, 'gain': 0.1460942501201363, 'value': None}
{'feature': None, 'threshold': None, 'data_left': None, 'data_right': None, 'gain': None, 'value': 1.0}
{'feature': None, 'threshold': None, 'data_

In [25]:
def predict_value(x, tree):
    if tree.value != None:
        return(tree.value)
    feature_value = x[tree.feature]
    #Go down left child
    if feature_value <= tree.threshold:
        return predict_value(x=x, tree=tree.data_left)

    #Go down right child
    if feature_value > tree.threshold:
        return predict_value(x=x, tree=tree.data_right) 

In [28]:
predict_value(X_np[0], my_tree)

0.0

(150, 4)

In [31]:
preds=[predict_value(X_np[i], my_tree) for i in range(0, X_np.shape[0])]

In [38]:
np.mean(y.values.flatten()==preds)

0.9933333333333333